Todo:

- navázat minulým řečníkem, když chybí

In [3]:
import os
import polars as pl

In [4]:
pl.Config.set_tbl_rows(50)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)
pl.Config.set_verbose(True)  

polars.config.Config

In [5]:
slovenske = [
    "S SK 1939",
    "S SR 1939-1945"
]

In [6]:
projevy = pl.read_ndjson(
    'data_raw/schuze/prepis_*.ndjson', 
    ignore_errors=True, 
    schema=pl.Schema(
        {
            'mluvci': pl.String, 
            'soubor': pl.String, 
            'mluvci_id': pl.String, 
            'text': pl.String, 
            'poradi': pl.Int32}
    ))

_init_credential_provider_builder(): credential_provider_init = None
_init_credential_provider_builder(): credential_provider_init = None


In [7]:
meta = pl.read_csv(
    'data_raw/schuze/meta_*.csv',
    schema=pl.Schema(
        {
            'komora': pl.String,
            'obdobi': pl.Int32,
            'schuze': pl.Int32,
            'cast': pl.Int32,
            'soubor': pl.String,
            'datum': pl.String,
            'komora_komplet': pl.String,
            'schuze_komplet': pl.String,
            'prepsano': pl.Boolean,
            'autorizovano': pl.Boolean
        }
    )
).with_columns(
    pl.col("datum").str.to_date(format="%Y-%m-%d",exact=False)
)

_init_credential_provider_builder(): credential_provider_init = None


In [8]:
try:
    os.makedirs("data")
except:
    pass

In [9]:
df = projevy.join(
    meta, on="soubor", how="left"
).filter(
    ~pl.col('komora_komplet').str.contains('SNR')
).filter(
    ~pl.col('komora_komplet').str.contains('NR SR')
).filter(
    ~pl.col('komora_komplet').is_in(slovenske)
).sort(
    by=['datum','poradi']
)

In [10]:
df.write_parquet("data/projevy.parquet", use_pyarrow=True)

In [11]:
len(df)

819251

In [12]:
len(meta)

102820

In [13]:
bez_data = meta.filter(pl.col("datum").is_null())
print(len(bez_data))

1738


In [14]:
bez_data.filter(pl.col("prepsano") == True)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,i32,i32,i32,str,date,str,str,bool,bool
"""ssr""",1939,11,2,"""1939ssr_stenprot_011schuz_s011002.htm""",null,"""S SR 1939-1945""","""11. schůze""",true,true
"""ssr""",1939,126,10,"""1939ssr_stenprot_126schuz_s126010.htm""",null,"""S SR 1939-1945""","""126. schůze""",true,true
"""ssr""",1939,128,2,"""1939ssr_stenprot_128schuz_s128002.htm""",null,"""S SR 1939-1945""","""128. schůze""",true,true
"""snr""",1948,2,2,"""1948snr_stenprot_002schuz_s002002.htm""",null,"""SNR 1948-1954""","""2. schůze""",true,true
"""snr""",1990,7,1,"""1990snr_stenprot_007schuz_s007001.htm""",null,"""SNR 1990-1992""","""7. schůze""",true,false
"""snr""",1990,7,2,"""1990snr_stenprot_007schuz_s007002.htm""",null,"""SNR 1990-1992""","""7. schůze""",true,false
"""snr""",1990,7,3,"""1990snr_stenprot_007schuz_s007003.htm""",null,"""SNR 1990-1992""","""7. schůze""",true,false
"""snr""",1990,7,4,"""1990snr_stenprot_007schuz_s007004.htm""",null,"""SNR 1990-1992""","""7. schůze""",true,false
"""snr""",1990,7,5,"""1990snr_stenprot_007schuz_s007005.htm""",null,"""SNR 1990-1992""","""7. schůze""",true,false


In [15]:
bez_data.sort(by='obdobi').tail(30)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,i32,i32,i32,str,date,str,str,bool,bool
"""ps""",2021,83,53,"""2021ps_stenprot_083schuz_s083053.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,54,"""2021ps_stenprot_083schuz_s083054.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,55,"""2021ps_stenprot_083schuz_s083055.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,56,"""2021ps_stenprot_083schuz_s083056.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,57,"""2021ps_stenprot_083schuz_s083057.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,58,"""2021ps_stenprot_083schuz_s083058.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,59,"""2021ps_stenprot_083schuz_s083059.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,60,"""2021ps_stenprot_083schuz_s083060.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2021,83,61,"""2021ps_stenprot_083schuz_s083061.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true


In [16]:
bez_komory = meta.filter(pl.col("komora_komplet").is_null())
print(len(bez_komory))

1002


In [17]:
bez_obdobi = meta.filter(pl.col("obdobi").is_null())
print(len(bez_obdobi))

0


In [18]:
bez_data = meta.filter(pl.col("datum").is_null()).sample(30)
print(len(bez_data))

30


In [19]:
meta.filter(pl.col("datum").is_null()).sample(30)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,i32,i32,i32,str,date,str,str,bool,bool
"""ps""",2021,83,74,"""2021ps_stenprot_083schuz_s083074.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false,true
"""ps""",2013,23,992,"""2013ps_stenprot_023schuz_s0231992.htm""",null,null,"""23. schůze""",false,null
"""ps""",2017,5,182,"""2017ps_stenprot_005schuz_s005182.htm""",null,null,"""4. schůze""",false,null
"""ps""",2013,23,551,"""2013ps_stenprot_023schuz_s023551.htm""",null,null,"""23. schůze""",false,null
"""ps""",2013,47,390,"""2013ps_stenprot_047schuz_s047390.htm""",null,null,"""47. schůze""",false,null
"""snr""",1990,8,32,"""1990snr_stenprot_008schuz_s008032.htm""",null,"""SNR 1990-1992""","""8. schůze""",true,false
"""snr""",1990,15,24,"""1990snr_stenprot_015schuz_s015024.htm""",null,"""SNR 1990-1992""","""15. schůze""",true,false
"""ps""",2013,23,143,"""2013ps_stenprot_023schuz_s0232143.htm""",null,null,"""23. schůze""",false,null
"""snr""",1990,24,26,"""1990snr_stenprot_024schuz_s024026.htm""",null,"""SNR 1990-1992""","""24. schůze""",true,false


In [20]:
import altair as alt

In [21]:
alt.Chart(
    df.filter(~pl.col('datum').is_null()).group_by_dynamic(index_column="datum", every="1y").agg(pl.col('text').len()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("text:Q")
)

alt.Chart(...)

In [22]:
df.filter(pl.col('datum').dt.year() == 1976)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Předseda ČNR Evžen Erban""","""1971cnr_stenprot_020schuz_s020002.htm""",null,"""Děkuji poslanci dr. Jiráskovi za přednesenou zprávu. O slovo se přihlásil místopředseda vlády ČSR Štěpán Horník. Prosím soudruha místopředsedu, aby se…",1,"""cnr""",1971,20,2,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
null,"""1971cnr_stenprot_020schuz_s020003.htm""",null,"""Z podnětu pracovní skupiny vlády ČSR jsme nastoupili alespoň začátek cesty přípravy porcovaného masa v kuchyňské úpravě. Není to jednoduché, chce to d…",1,"""cnr""",1971,20,3,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
"""Předseda ČNR Evžen Erban""","""1971cnr_stenprot_020schuz_s020004.htm""",null,"""Děkuji soudruhu místopředsedovi vlády Horníkovi za přednesenou zprávu vlády. Soudružky a soudruzi, v závěru našeho jednání k tomuto bodu přijme Česká …",1,"""cnr""",1971,20,4,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
"""Místopředseda ČNR Věroslav Jedlička""","""1971cnr_stenprot_020schuz_s020005.htm""",null,"""Děkuji poslankyni Hausnerové, slovo má poslanec Jirsík.""",1,"""cnr""",1971,20,5,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
"""Místopředseda ČNR JUDr. Vladimír Ambruz""","""1971cnr_stenprot_020schuz_s020008.htm""",null,"""Děkuji. Dávám slovo poslankyni Hlaváčkové.""",1,"""cnr""",1971,20,8,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
null,"""1971cnr_stenprot_020schuz_s020001.htm""",null,"""/Začátek schůze v 9.30 hodin/ Přítomni: Předseda České národní rady Evžen Erban a místopředsedové České národní rady Oldřich Voleník, JUDr. Vladimír A…",1,"""cnr""",1971,20,1,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
"""Místopředseda ČNR JUDr. Vladimír Ambruz""","""1971cnr_stenprot_020schuz_s020007.htm""",null,"""Děkuji poslankyni Kostkové. O slovo se přihlásil ministr zemědělství a výživy Nágr.""",1,"""cnr""",1971,20,7,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
"""Místopředseda ČNR Oldřich Voleník""","""1971cnr_stenprot_020schuz_s020006.htm""",null,"""Děkuji soudruhu ministru Prokopcovi. Slovo má poslanec Miler.""",1,"""cnr""",1971,20,6,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
"""Předseda ČNR Evžen Erban""","""1971cnr_stenprot_020schuz_s020009.htm""",null,"""A nyní z pověření vlády České socialistické republiky odůvodní předložený vládní návrh ministr vnitra ing. Josef Jung. Prosím soudruha ministra, aby s…",1,"""cnr""",1971,20,9,1976-03-23,"""ČNR 1971-1976""","""20. schůze""",true,true
